In [1]:
import sys
sys.path.append('..')
sys.path.append('../..')

In [2]:
from os.path import join as pjoin

Here for this blog post, I will be showing you how to implement a RBAC (Role-Based Access Control) with the help of JWT in Qdrant Vector Database.

For this purpose, I will be using the following data structure for creating multiple collections.
```
├── data
│   ├── financial
│   │   └── Sample-Accounting-Income-Statement-PDF-File.pdf
│   └── general
│       ├── avengers-endgame-script-pdf.pdf
│       └── security_policy.pdf
```
The idea is to create two collections, one for financial data and the other for general data. General data will have multiple files, and financial data will have only one file. Then we will see RBAC in action and see how we can restrict access of user based on the role assigned to them.

In [39]:
# Qdrant related Parameters
api = 'jhvfegfeboihf313fekfgejbv' # 'your_api_key'
host = 'localhost'
port = 6333
url = f'http://{host}:{port}'

I will be keeping the loading of the dataset, generating embeddings, and creating collections at a very high level. If you want to know more about it, you can refer to one of my previous blog posts [here](https://quamernasim.medium.com/hindi-language-ai-chatbot-for-enterprises-using-llama-3-qdrant-ollama-langchain-and-mlflow-9b69391d3348) where I have explained how to build a chatbot using Qdrant, Llama3, Ollama, and Langchain. For this blog post, I will be focusing on RBAC.

Let's start with different security senarios.

# 1. Without any token with RBAC enabled

We have enabled RBAC in the Qdrant, but we have not created any tokens yet. Let's see how it behaves in this case.

In [7]:
from qdrant_client import QdrantClient

client = QdrantClient(url=url)
client.get_collections()

UnexpectedResponse: Unexpected Response: 401 (Unauthorized)
Raw response content:
b'Must provide an API key or an Authorization bearer token'

As expected, it should not allow any operation without a token. Now in the next section let's create a JWT token and try to access the Qdrant API.

# 2. Global Read-Ony Access

Here let's first create a token with global read-only access. With global read-only access, the user can only read the resources in the cluster. They cannot create, update, or delete resources. This essentially means that the user can read all the collections available, so be careful when granting this permission.

In [9]:
import time
from utils import generate_jwt

current_time = int(time.time())

# This payload along with the API is used to generate the JWT token.
# This token tells that the user has global read only access to all the collections.
# It also specifies that this token will expire in 1 hour.
payload = {
  "access": "r",
  "exp": current_time + 3600, # 1 hour
}

# Generate the JWT token
# This token will be used to authenticate the user.
jwt = generate_jwt(api, payload)

Currently, we have no collections in the Qdrant Vector DB. Let's see how the API behaves in this case.

In [10]:
from qdrant_client import QdrantClient

client = QdrantClient(url=url, api_key=jwt)
client.get_collections()

/opt/conda/lib/python3.10/site-packages/qdrant_client/qdrant_remote.py:130: UserWarning: Api key is used with an insecure connection.
  warnings.warn("Api key is used with an insecure connection.")


CollectionsResponse(collections=[])

Great, it returned an empty list of collections. Now let's try to create a collection with the same token. Note that this should fail as the token has only read-only access.

In [11]:
from qdrant_client import QdrantClient

client = QdrantClient(url=url, api_key=jwt)
# Delete the collection if it exists
client.delete_collection(collection_name=collection_name)

UnexpectedResponse: Unexpected Response: 403 (Forbidden)
Raw response content:
b'{"status":{"error":"Forbidden: Global manage access is required"},"time":0.000023168}'

As we can see, the API returned a 403 Forbidden error saying 'Global manage access is required' to create a collection.

Now in the next section, let's create a token with global manage access and try to create a collection.

# 3. Global Manage Access

Now let's create a token with global manage access. With Global Manage Access, the user can read, create, update, and delete collections in the cluster. This essentially means that the user can perform all the operations on all the collections available, so be extremely careful when granting this permission. You should only grant this permission to Admins.

In [34]:
import time
from utils import generate_jwt

current_time = int(time.time())

# This payload along with the API is used to generate the JWT token.
# This token tells that the user has global manage access to all the collections.
# It also specifies that this token will expire in 1 hour.
# You should only generate this token for admin users. 
payload = {
  "access": "m",
  "exp": current_time + 3600, # 1 hour
}

# Generate the JWT token
# This token will be used to authenticate the user.
jwt = generate_jwt(api, payload)

Now let's again try to list the collections using this new token.

In [ ]:
from qdrant_client import QdrantClient

client = QdrantClient(url=url, api_key=jwt)
client.get_collections()

/opt/conda/lib/python3.10/site-packages/qdrant_client/qdrant_remote.py:130: UserWarning: Api key is used with an insecure connection.
  warnings.warn("Api key is used with an insecure connection.")


CollectionsResponse(collections=[])

Since no collections are available, it returned an empty list. Now let's try to delete a collection using this token. Remember, this operation failed using the previous read-only token. Even though we don't have any collections, let's try to delete a collection and see what happens.

In [37]:
from qdrant_client import QdrantClient

client = QdrantClient(url=url, api_key=jwt)
# Delete the collection if it exists
client.delete_collection(collection_name=collection_name)

/opt/conda/lib/python3.10/site-packages/qdrant_client/qdrant_remote.py:130: UserWarning: Api key is used with an insecure connection.
  warnings.warn("Api key is used with an insecure connection.")


False

As we can see, it ran successfully and returned False as there was no collection to delete. Now let's try to create two collections, one for financial data and the other for general data. Then we will try to explore the RBAC in more detail.

Before creating the collections, let's first load the embeddings model to generate embeddings for the documents. Here we will keep the indexing phase on a high level. If you're interested in knowing more about it, you can refer to one of my previous blog posts [here](https://quamernasim.medium.com/hindi-language-ai-chatbot-for-enterprises-using-llama-3-qdrant-ollama-langchain-and-mlflow-9b69391d3348). For this blog post, I will be focusing on RBAC.

In [ ]:
import fasttext as ft
# download the fasttext model from https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz and then unzip it
embedding_model_path = '/home/quamer23nasim38/Role-Based-Access-Control-of-Qdrant-Vector-Database/embedding_model/cc.en.300.bin'
# Load the embedding model
embed_model = ft.load_model(embedding_model_path)

Then let's define some constants for the creation of collections.

In [38]:
# data related parameters
chunk_size = 500
chunk_overlap = 50
batch_size = 4000

# vector related parameters
vector_size = 300

Let's create the chunk out of the general data

In [ ]:
from langchain_community.document_loaders import DirectoryLoader
# from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

collection_type = 'general'
root = '/home/quamer23nasim38/Role-Based-Access-Control-of-Qdrant-Vector-Database/data'
data_path = pjoin(root, collection_type)
collection_name = collection_type

# Load the documents from the directory
loader = DirectoryLoader(data_path, loader_cls=PyPDFLoader)

# Split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    length_function=len,
    is_separator_regex=False,
)
docs = loader.load_and_split(text_splitter=text_splitter)

from utils import generate_embeddings_from_fastext_model
# Generate the embeddings for the data
df = generate_embeddings_from_fastext_model(docs, embed_model)

from utils import create_new_collection
# Create a new collection with manage access
create_new_collection(url, jwt, collection_name, df, vector_size, batch_size, delete_prev = True, create_from_scratch = True)

We do the same for the financial data.

In [ ]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

collection_type = 'financial'
root = '/home/quamer23nasim38/Role-Based-Access-Control-of-Qdrant-Vector-Database/data'
data_path = pjoin(root, collection_type)
collection_name = collection_type

# Load the documents from the directory
loader = DirectoryLoader(data_path, loader_cls=PyPDFLoader)

# Split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    length_function=len,
    is_separator_regex=False,
)
docs = loader.load_and_split(text_splitter=text_splitter)

from utils import generate_embeddings_from_fastext_model
# Generate the embeddings for the data
df = generate_embeddings_from_fastext_model(docs, embed_model)

from utils import create_new_collection
# Create a new collection with manage access
create_new_collection(url, jwt, collection_name, df, vector_size, batch_size, delete_prev = True, create_from_scratch = True)

Great! Now we have created two collections, 'general' and 'financial' collections. Let's see if we can read these collections with different sets of tokens having different permissions.

# 4. Collection Specific Access

With this access, we can limit the access of the user to a specific collection only. This is the most secure way to grant access to the user. We can also limit the access to the types of documents in that collection or pages as well. Let's see how we can do this.

In our Qdrant Vector Database, we now have two collections, 'general' and 'financial'. As can be understood from the names, the 'general' collection contains general data, and the 'financial' collection contains financial data. And due to the nature of the data, we want to restrict access of each user to specific collections as per their roles in the organization.

## 4.1 Read-Only Access

Here in this section we will create a token with read-only access to a specific collection only. Let's see how it behaves.

In [10]:
import time
from utils import generate_jwt

current_time = int(time.time())

# This payload along with the API is used to generate the JWT token.
# This token tells that the user has read access to the general collection only.
# You can give access to multiple collections by adding multiple dictionaries in the access list.
# For now, we are only giving access to the general collection.
# It also specifies that this token will expire in 1 hour.
payload = {
  "exp": current_time + 3600, # 1 hour
  "access": [
    {
      "collection": "general", # collection name - Change this to the collection you want to give access to, like financial
      "access": "r"
    },
  ]
}

# Generate the JWT token
# This token will be used to authenticate the user.
jwt = generate_jwt(api, payload)

In [9]:
from qdrant_client import QdrantClient

client = QdrantClient(url=url, api_key=jwt)
client.get_collections()

/opt/conda/lib/python3.10/site-packages/qdrant_client/qdrant_remote.py:130: UserWarning: Api key is used with an insecure connection.
  warnings.warn("Api key is used with an insecure connection.")


CollectionsResponse(collections=[CollectionDescription(name='general')])

This is great. We can see that the user can only access the 'general' collection and not the 'financial' collection. Now let's try to verify if the user has read-only access to the 'general' collection.

In [18]:
import numpy as np

# We are generating a random query vector of size vector_size
query_vector = np.random.rand(vector_size)

# We are searching for the closest points to the query vector in the general collection
# Since we have the read access to the general collection, we can search in it.
hits = client.search(
   collection_name="general",
   query_vector=query_vector,
   limit=5  # Return 5 closest points
)
hits

[ScoredPoint(id=11, version=2, score=0.07114598, payload={'metadata': {'page': 1, 'source': '/home/quamer23nasim38/Role-Based-Access-Control-of-Qdrant-Vector-Database/data/general/security_policy.pdf'}, 'page_content': 'is built using the Rust language - a static multi-paradigm, memory-efficient, low-level programming language focused on speed, security, and performance. The intention is to build Qdrant with as few moving parts as possible, thereby keeping the attack vector as low as possible. Email security Qdrant supports TLS encryption on all inbound and outbound emails. Qdrant uses Gmail to provide email and communication services. For an explanation of how email encryption works, take a look at this'}, vector=None, shard_key=None),
 ScoredPoint(id=10, version=2, score=0.045524757, payload={'metadata': {'page': 1, 'source': '/home/quamer23nasim38/Role-Based-Access-Control-of-Qdrant-Vector-Database/data/general/security_policy.pdf'}, 'page_content': 'within the Qdrant Cloud platform

As we can see, the user can read the 'general' collection. Now let's try to update the 'general' collection with the same read-only token. Let's hope that it fails.

In [21]:
import numpy as np
from qdrant_client.models import PointStruct

# We are generating 100 random vectors of size vector_size
vectors = np.random.rand(100, vector_size)

# We are upserting these vectors in the general collection
# Since we have the read only access to the general collection, we won't be able to upsert the vectors.
client.upsert(
   collection_name="general",
   points=[
      PointStruct(
            id=idx,
            vector=vector.tolist(),
            payload={"color": "red", "rand_number": idx % 10}
      )
      for idx, vector in enumerate(vectors)
   ]
)

UnexpectedResponse: Unexpected Response: 403 (Forbidden)
Raw response content:
b'{"status":{"error":"Forbidden: Write access to collection general is required"},"time":0.000079842}'

As we guessed, the API returned a 403 Forbidden error saying 'Write access to collection general is required'. This is great. Now let's see if we can read the financial collection with the same read-only token for the general collection. Just a heads up, this should fail.

In [24]:
# We are generating a random query vector of size vector_size
query_vector = np.random.rand(vector_size)

# We are searching for the closest points to the query vector in the general collection
# Since we have the read access to the general collection only, we won't be able to search in financial collection.
hits = client.search(
   collection_name="financial",
   query_vector=query_vector,
   limit=5  # Return 5 closest points
)
hits

UnexpectedResponse: Unexpected Response: 403 (Forbidden)
Raw response content:
b'{"status":{"error":"Forbidden: Access to collection financial is required"},"time":7.61e-6}'

Great. Once again, the API returned a 403 Forbidden error saying 'Access to collection financial is required'. Now in the next section, let's test with read-write access to a specific collection. In the next section we will also see how we can grant access to multiple collections to a user.

## 4.2 Read-Writes Access

Here we will grant the user read-write access to the 'general' collection only. And on top of that, we will limit the access of the 'financial' collection to read-only.

In [12]:
import time
from utils import generate_jwt

current_time = int(time.time())

# This payload along with the API is used to generate the JWT token.
# This token tells that the user access to two collections, general and financial.
# The access to the general collection is read-write and the access to the financial collection is read only.
# It also specifies that this token will expire in 1 hour.
payload = {
  "exp": current_time + 3600, # 1 hour
  "access": [
    {
      "collection": "general",
      "access": "rw"
    },
    {
      "collection": 'financial',
      "access": "r"
    }
  ]
}

# Generate the JWT token
# This token will be used to authenticate the user.
jwt = generate_jwt(api, payload)

In [14]:
from qdrant_client import QdrantClient
client = QdrantClient(url=url, api_key=jwt)

collection = client.get_collections()
collection

/opt/conda/lib/python3.10/site-packages/qdrant_client/qdrant_remote.py:130: UserWarning: Api key is used with an insecure connection.
  warnings.warn("Api key is used with an insecure connection.")


CollectionsResponse(collections=[CollectionDescription(name='general'), CollectionDescription(name='financial')])

Nice! The user has now access to both the collections, 'general' and 'financial'. Now let's try to update the 'general' collection with the same token. Since the token has read-write access to the 'general' collection, it should work.

In [29]:
import numpy as np
from qdrant_client.models import PointStruct

# We are generating 100 random vectors of size vector_size
vectors = np.random.rand(100, vector_size)

# We are upserting these vectors in the general collection
# Since we have the read-write access to the general collection, we can upsert the vectors.
client.upsert(
   collection_name="general",
   points=[
      PointStruct(
            id=idx,
            vector=vector.tolist(),
            payload={"color": "red", "rand_number": idx % 10}
      )
      for idx, vector in enumerate(vectors)
   ]
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

Looks good. We can see that the user can update the 'general' collection. Now let's try to update the 'financial' collection with the same token. This should fail as the token has only read-only access to the 'financial' collection.

In [30]:
import numpy as np
from qdrant_client.models import PointStruct

vectors = np.random.rand(100, vector_size)
client.upsert(
   collection_name="financial",
   points=[
      PointStruct(
            id=idx,
            vector=vector.tolist(),
            payload={"color": "red", "rand_number": idx % 10}
      )
      for idx, vector in enumerate(vectors)
   ]
)

UnexpectedResponse: Unexpected Response: 403 (Forbidden)
Raw response content:
b'{"status":{"error":"Forbidden: Write access to collection financial is required"},"time":0.000062905}'

As expected, the API returned a 403 Forbidden error saying 'Write access to collection financial is required'. Now let's try do a final check for this token. Let's see if the user can read the 'financial' collection.

In [15]:
# We are generating a query vector from a string that was available in one of the documents in the general collection
x = "based on the equation: assets = liabilities + owners' equity."
query_vector = embed_model.get_sentence_vector(x).tolist()

# We are searching for the closest points to the query vector in the general collection
# Since we have the read-write access to the general collection, we can search in it.
hits = client.search(
   collection_name="financial",
   query_vector=query_vector,
   limit=20  # Return 5 closest points
)
hits

[ScoredPoint(id=3, version=0, score=0.7493207, payload={'metadata': {'page': 0, 'source': '/home/quamer23nasim38/Role-Based-Access-Control-of-Qdrant-Vector-Database/data/financial/Sample-Accounting-Income-Statement-PDF-File.pdf'}, 'page_content': "Balance Sheet The balance sheet is based on the equation: assets = liabilities + owners' equity . It indicates everything the company owns (assets), everything the company owes to creditors (liabilities) and the value of the ownership stake in the company (shareholders' equity, or capital). The balance sheet date is the ending date of the period or year, and is a continuation of the amounts recorded since the"}, vector=None, shard_key=None),
 ScoredPoint(id=8, version=0, score=0.7304637, payload={'metadata': {'page': 0, 'source': '/home/quamer23nasim38/Role-Based-Access-Control-of-Qdrant-Vector-Database/data/financial/Sample-Accounting-Income-Statement-PDF-File.pdf'}, 'page_content': 'period. Sources of cash listed on the statement include re

Great! The user can read the 'financial' collection. Now in the next section, let's go ahead and see how we can even limit the user access within single collection. We basically want to limit the access to specific types of documents in the collection.

# 5. Document Specific Access

Now in this last section, we will limit the access of the user to specific types of documents in the collection. This is one of the most secure ways to grant access to the user. Assume a scenario where you have a collection of 'general' data and in that collection, you have multiple types of documents, you can limit the access of the user to specific types of documents only. Let's see how we can do this.

Here we create a token that allowed the user to access only the general collection and only the documents which named 'security_policy.pdf'. 

In [19]:
import time
from utils import generate_jwt

current_time = int(time.time())

# This payload along with the API is used to generate the JWT token.
# This token tells that the user has access to the general collection only.
# The access to the general collection is read-write.
# It also specifies that the token only limits the access to the document security_policy.pdf in the general collection.
# It also specifies that this token will expire in 1 hour.
payload = {
  "exp": current_time + 3600, # 1 hour
  "access": [
    {
      "collection": "general",
      "access": "rw",
      "payload": {
        "metadata.source": "/home/quamer23nasim38/Role-Based-Access-Control-of-Qdrant-Vector-Database/data/general/security_policy.pdf",
      }
    },
  ]
}

# Generate the JWT token
jwt = generate_jwt(api, payload)

In [21]:
from qdrant_client import QdrantClient

client = QdrantClient(url=url, api_key=jwt)

# We are generating a query vector from a string that was available in the document security_policy.pdf in the general collection
x = 'take the security of Qdrant code'
query_vector = embed_model.get_sentence_vector(x).tolist()

# We are searching for the closest points to the query vector in the general collection
hits = client.search(
   collection_name="general",
   query_vector=query_vector,
   limit=5  # Return 5 closest points
)
hits

[ScoredPoint(id=10, version=2, score=0.8121032, payload={'metadata': {'page': 1, 'source': '/home/quamer23nasim38/Role-Based-Access-Control-of-Qdrant-Vector-Database/data/general/security_policy.pdf'}, 'page_content': 'within the Qdrant Cloud platform, and only the necessary ports are opened on each server. All outbound connections pass through the stateless access control rules, whilst inbound connections from the internet must pass through a secure, highly-available load balancer layer, and the stateless access control firewall rules before then being routed to each server. Software security We take the security of the Qdrant code very seriously. The database is built using the Rust language - a static'}, vector=None, shard_key=None),
 ScoredPoint(id=5, version=2, score=0.7998578, payload={'metadata': {'page': 0, 'source': '/home/quamer23nasim38/Role-Based-Access-Control-of-Qdrant-Vector-Database/data/general/security_policy.pdf'}, 'page_content': 'into account the impact of company 

As we can see, the search query returned only the chunks of the document 'security_policy.pdf'. It did not return any other documents. Now let's try to go even further and limit the access to a specific page of the document. Let's see how we can do this.

Here on top of all the previous restrictions, we have also limited the access to the second page of the document 'security_policy.pdf'. Let's see if the user can access any other page except the second page of the document.

In [22]:
import time
from utils import generate_jwt

current_time = int(time.time())

# This payload along with the API is used to generate the JWT token.
# This token tells that the user has access to the general collection only.
# The access to the general collection is read-write.
# It also specifies that the token only limits the access to the document security_policy.pdf in the general collection.
# It also specifies that the token only limits the access to the second page (page index starts with 0) of the document.
# It also specifies that this token will expire in 1 hour.
payload = {
  "exp": current_time + 3600, # 1 hour
  "access": [
    {
      "collection": "general",
      "access": "rw",
      "payload": {
        "metadata.source": "/home/quamer23nasim38/Role-Based-Access-Control-of-Qdrant-Vector-Database/data/general/security_policy.pdf",
        "metadata.page": 1
      }
    },
  ]
}

# Generate the JWT token
jwt = generate_jwt(api, payload)

In [23]:
from qdrant_client import QdrantClient

client = QdrantClient(url=url, api_key=jwt)

x = 'take the security of Qdrant code'
query_vector = embed_model.get_sentence_vector(x).tolist()

hits = client.search(
   collection_name="general",
   query_vector=query_vector,
   limit=20  # Return 5 closest points
)
hits

[ScoredPoint(id=10, version=2, score=0.8121032, payload={'metadata': {'page': 1, 'source': '/home/quamer23nasim38/Role-Based-Access-Control-of-Qdrant-Vector-Database/data/general/security_policy.pdf'}, 'page_content': 'within the Qdrant Cloud platform, and only the necessary ports are opened on each server. All outbound connections pass through the stateless access control rules, whilst inbound connections from the internet must pass through a secure, highly-available load balancer layer, and the stateless access control firewall rules before then being routed to each server. Software security We take the security of the Qdrant code very seriously. The database is built using the Rust language - a static'}, vector=None, shard_key=None),
 ScoredPoint(id=8, version=2, score=0.76660895, payload={'metadata': {'page': 1, 'source': '/home/quamer23nasim38/Role-Based-Access-Control-of-Qdrant-Vector-Database/data/general/security_policy.pdf'}, 'page_content': 'All servers are tested for vulnera

As we can see, the exact same search query like last section returned only the second page of the document 'security_policy.pdf' as expected.

Now before I end this tutorial, let me show you one more way to limit the access of the user. Here we will limit the access of the user by using the 'value_exists' filter. This basically means that the user can only access the collection if the specific field exists in the document. Though this can be extended to several use cases, like user_id, user_role, etc, but for the ease of this tutorial, let's just use the 'value_exists' filter to check the presence of the document type. If the document type exists, then only the user can access the collection.

let's first see what happens if the document type does not exist in the document.

In [30]:
import time
from utils import generate_jwt

current_time = int(time.time())

# This payload along with the API is used to generate the JWT token.
# This token tells that the user has access to the general collection only.
# The access to the general collection is read-write.
# Apart from the access to read-write the general collection, the token also specifies to check
# if the metadata.source key in the document matches the value "/home/quamer23nasim38/Role-Based-Access-Control-of-Qdrant-Vector-Database/data/general/avengers-endgame-script-pdf.pdf".
# if it matches, then the user will have read-write access to the general collection.
# if it doesn't match, then the user won't have any access to the general collection.
# It also specifies that this token will expire in 1 hour.
payload = {
  "exp": current_time + 3600, # 1 hour
  "value_exists": {
    "collection": "general",
    "matches": [
      {
          "key": "metadata.source", 
          "value": "/home/quamer23nasim38/Role-Based-Access-Control-of-Qdrant-Vector-Database/data/general/blah blah blah.pdf"
      }
    ]
  },
  "access": [
    {
      "collection": "general",
      "access": "rw",
    },
  ]
}

# Generate the JWT token
jwt = generate_jwt(api, payload)

In [31]:
from qdrant_client import QdrantClient

client = QdrantClient(url=url, api_key=jwt)

# We are generating a query vector from a string that was available in the document avengers-endgame-script-pdf.pdf in the general collection
x = 'take the security of Qdrant code'
query_vector = embed_model.get_sentence_vector(x).tolist()

hits = client.search(
   collection_name="general",
   query_vector=query_vector,
   limit=5  # Return 5 closest points
)
hits

UnexpectedResponse: Unexpected Response: 401 (Unauthorized)
Raw response content:
b'Invalid JWT, stateful validation failed'

Woa! The API returned a 401 Unauthorized error. It clearly says that the validation failed! This is great. Now let's see what happens if the document type exists in the document.

In [32]:
import time

current_time = int(time.time())

# This payload along with the API is used to generate the JWT token.
# This token tells that the user has access to the general collection only.
# The access to the general collection is read-write.
# Apart from the access to read-write the general collection, the token also specifies to check
# if the metadata.source key in the document matches the value "/home/quamer23nasim38/Role-Based-Access-Control-of-Qdrant-Vector-Database/data/general/avengers-endgame-script-pdf.pdf".
# if it matches, then the user will have read-write access to the general collection.
# if it doesn't match, then the user won't have any access to the general collection.
# It also specifies that this token will expire in 1 hour.
payload = {
  "exp": current_time + 3600, # 1 hour
  "value_exists": {
    "collection": "general",
    "matches": [
      {
          "key": "metadata.source", 
          "value": "/home/quamer23nasim38/Role-Based-Access-Control-of-Qdrant-Vector-Database/data/general/avengers-endgame-script-pdf.pdf"
      }
    ]
  },
  "access": [
    {
      "collection": "general",
      "access": "rw",
    },
  ]
}

# Generate the JWT token
jwt = generate_jwt(api, payload)

In [33]:
client = QdrantClient(url=url, api_key=jwt)

x = 'take the security of Qdrant code'
query_vector = embed_model.get_sentence_vector(x).tolist()

hits = client.search(
   collection_name="general",
   query_vector=query_vector,
   limit=5  # Return 5 closest points
)
hits

/opt/conda/lib/python3.10/site-packages/qdrant_client/qdrant_remote.py:130: UserWarning: Api key is used with an insecure connection.
  warnings.warn("Api key is used with an insecure connection.")


[ScoredPoint(id=10, version=2, score=0.8121032, payload={'metadata': {'page': 1, 'source': '/home/quamer23nasim38/Role-Based-Access-Control-of-Qdrant-Vector-Database/data/general/security_policy.pdf'}, 'page_content': 'within the Qdrant Cloud platform, and only the necessary ports are opened on each server. All outbound connections pass through the stateless access control rules, whilst inbound connections from the internet must pass through a secure, highly-available load balancer layer, and the stateless access control firewall rules before then being routed to each server. Software security We take the security of the Qdrant code very seriously. The database is built using the Rust language - a static'}, vector=None, shard_key=None),
 ScoredPoint(id=5, version=2, score=0.7998578, payload={'metadata': {'page': 0, 'source': '/home/quamer23nasim38/Role-Based-Access-Control-of-Qdrant-Vector-Database/data/general/security_policy.pdf'}, 'page_content': 'into account the impact of company 

Nice! It validated the document type and confirmed that the document type exists in the document. Once the validation is successful, it returned the chunks of the document from the correct document irrespective of the validation document type.

Finally, we have seen how we can use the Qdrant Vector Database with RBAC enabled and how we can grant access to the user based on their roles. We have seen how we can grant global read-only access, global manage access, collection-specific access, and document-specific access. We have also seen how we can limit the access of the user by using the 'value_exists' filter. This is a very powerful feature of the Qdrant Vector Database and can be used in various use cases.